In [1]:
catagories = {'C01':'Bacterial Infections and Mycoses',
'C02':'Virus Diseases',
'C03':'Parasitic Diseases',
'C04':'Neoplasms',
'MusCuloskeletal Diseases':'C05',
'Digestive System Diseases':'C06',
'Stomatognathic Diseases':'C07',
'Respiratory Tract Diseases':'C08',
'Otorhinolaryngologic Diseases':'C09',
'Nervous System Diseases':'C10',
'Eye Diseases':'C11',
'Urologic and Male Genital Diseases':'C12',
'Female Genital Diseases and Pregnancy Complications':'C13',
'CardiovasCular Diseases':'C14',
'Hemic and Lymphatic Diseases':'C15',
'Neonatal Diseases and Abnormalities':'C16',
'Skin and Connective Tissue Diseases':'C17',
'Nutritional and Metabolic Diseases':'C18',
'Endocrine Diseases':'C19',
'Immunologic Diseases':'C20',
'Disorders of Environmental Origin':'C21',
'Animal Diseases':'C22',
'Pathological Conditions, Signs and Symptoms':'C23'}

In [2]:
from __future__ import print_function

import logging
import numpy as np
import sys
from time import time
import matplotlib.pyplot as plt

from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

from sklearn.utils.extmath import density
from sklearn import metrics

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
dataset_folder = "./ohsumed-first-20000-docs"

In [4]:
training_folder = dataset_folder + '/training'
test_folder = dataset_folder + '/test'

In [5]:
test_data = load_files(container_path=test_folder, categories=None, load_content=True, shuffle=True, encoding='utf-8', random_state=17)
training_data = load_files(container_path=training_folder, categories=None, load_content=True, shuffle=True, encoding='utf-8', random_state=17)

In [6]:
categories = training_data.target_names

In [7]:
print("categories values: ",categories)

categories values:  ['C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23']


In [25]:
y_train,y_test = training_data.target , test_data.target

In [26]:
import nltk
import string
from multiprocessing import Pool
from itertools import chain
nltk.download('punkt')
punkt = nltk.data.load('tokenizers/punkt/english.pickle')
flatten = lambda l: [item for sublist in l for item in sublist]
catagory_stats = {}
for train_data_file, target in zip(training_data.data, training_data.target):
    sentences = punkt.tokenize(train_data_file.lower())
    words = flatten((reduce(lambda x, y: x.append(nltk.tokenize.word_tokenize(y)) or x , sentences, [])))
    words = [word for word in words if not all(char in string.punctuation for char in word)]
    words = [word for word in words if not (word.startswith("'") and len(word) <=2)]
    if target in catagory_stats:
        catagory_stats[target].append(words)
    else:
        catagory_stats[target] = words

[nltk_data] Downloading package punkt to /Users/arnon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
from collections import Counter
catagory_stats_counting_words = {}
stopwords = nltk.corpus.stopwords.words('english')

for catagory, words in catagory_stats.iteritems():
    counted_words = Counter(words)
    counted_words_stopwords_remove = counted_words.copy()
    for key in counted_words:
        if key in stopwords:
            del counted_words_stopwords_remove[key]
    catagory_stats_counting_words[catagory] = counted_words_stopwords_remove

IndentationError: unindent does not match any outer indentation level (<ipython-input-27-7fe2a4ca3ac3>, line 7)

In [ ]:
for catagory, words in catagory_stats_counting_words.iteritems():
    print(catagory)
    print(words.most_common(n=10))